# INPUT

In [ ]:
!pip install -r requirements.txt

In [9]:
###  INPUT ###
import pandas as pd
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')

# 모델링 시작

In [10]:
#    Write codes...
# ... (Data preprocessing code here)

#데이터셋 로드
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')


In [11]:
# 모델링 준비
import math
import numpy as np
#from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

# 파라미터 튜닝
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# 대충 돌릴때 체크 : cross-val
from sklearn.model_selection import cross_val_score

# 평가
from sklearn.metrics import accuracy_score

In [12]:

# Split the data into training and testing sets

# 데이터 분할 (훈련 세트와 테스트 세트)
X = input_data[['ec', 'cunt', 'HeatingEnergyUsage_cumsum', 'hvstCo', 'frtstCo', 'HeatingEnergyUsage_cumsum']] # 초기에 진행했던 데이터


#X = input_data[['ec', 'cunt', 'HeatingEnergyUsage_cumsum', 'hvstCo', 'frtstCo', 'HeatingEnergyUsage_cumsum', 'inTp', 'inHd', 'inCo2', 'acSlrdQy',
 #              'grwtLt', 'frtstSetCo', 'hvstCo', 'FertilizerUsage', 'CO2Usage', 'MistUsageTime']]
y = input_data[['outtrn_cumsum']]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 현재최대 - modeling 파일 참고

In [29]:
#딥러닝 테스트
# 랜덤 시드 고정
SEED = 2021
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 데이터 전처리
from sklearn import datasets

#housing = datasets.load_boston()
#X_data = housing.data
#y_data = housing.target
X_data = X
y_data = y
X_data.shape
#덮어쓰기

(84840, 6)

In [14]:
# 피처 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_data_scaled = scaler.fit_transform(X_data)

X_data_scaled[0]

# 학습 - 테스트 데이터셋 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, shuffle=True, random_state = SEED)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



(67872, 6) (67872, 1)
(16968, 6) (16968, 1)


In [ ]:
# MLP 모델 아키텍처 정의

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

def build_model(num_input=1):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=num_input))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='linear'))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    return model

model = build_model(num_input=6)  #num_input에 input 피처의 개수를 넣어줘야 함

# 미니 배치 학습
model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=2)

# 모델 평가
model.evaluate(X_test, y_test)

#시간 01:27 실행시작 - 01:37 종료

In [33]:
# 교차 검증
model = build_model(num_input=6)  #num_input에 input 피처의 개수를 넣어줘야 함
history = model.fit(X_train, y_train, batch_size=32, epochs=200, validation_split=0.25, verbose=2)

#
import matplotlib.pyplot as plt

def plot_loss_curve(total_epoch=10, start=1):
    plt.figure(figsize=(15, 5))
    plt.plot(range(start, total_epoch +1), history.history['loss'][start-1:total_epoch], label='Train')
    plt.plot(range(start, total_epoch +1), history.history['val_loss'][start-1:total_epoch], label='Validation')
    plt.xlabel('Epochs')
    plt.ylabel('mse')
    plt.legend()
    plt.show()

plot_loss_curve(total_epoch=200, start=1)

plot_loss_curve(total_epoch=200, start=20)
# 시간 01:37 실행시작 01:45 종료

Epoch 1/200


KeyboardInterrupt: ignored

In [40]:
#예측하기
deep_predictions = model.predict(X_test)

531/531 [==============================] - 1s 972us/step


In [41]:
# Calculate RMSE between the predictions and actual 'y' values
def calculate_rmse(targets, predictions):
    """
    Calculate the Root Mean Squared Error (RMSE) between predicted and target values.

    :param predictions: Predicted values.
    :type predictions: array-like
    :param targets: Target values.
    :type targets: array-like
    :return: RMSE value.
    :rtype: float
    """
    return np.sqrt(mean_squared_error(targets, predictions)) # 가장 큰 값 리턴
    #from sklearn.metrics import mean_squared_error
    #return np.sqrt(mean_squared_error(targets, predictions))


# Calculate r2_score between the predictions and actual 'y' values
def calculate_R2_score(y_test,y_pred):
    return r2_score(y_test, y_pred) # 가장 큰값 리턴
    #from sklearn.metrics import r2_score
    #return r2_score(y_test, y_pred)

rmse = calculate_rmse(y_test, deep_predictions)
r2score = calculate_R2_score(y_test, deep_predictions)
# RMSE와 R^2 스코어 평가



In [42]:
### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)


RMSE: 35029.879400575956
R2_score: 0.19842321387298434
